In [1]:
#PyDrive 설치
!pip install PyDrive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import from_numpy, tensor, nn, cuda, optim
from torch.utils import data
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import time
from google.colab import files

#https://buomsoo-kim.github.io/colab/2018/04/16/Importing-files-from-Google-Drive-in-Google-Colab.md/
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd


manualSeed = 1
 
np.random.seed(manualSeed)
#random.seed(manualSeed)
torch.manual_seed(manualSeed)
'''
# if you are using GPU
torch.cuda.manual_seed(manualSeed)
torch.cuda.manual_seed_all(manualSeed)

torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
'''
num_workers = 0
torch.backends.cudnn.enabled = True

In [3]:

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [4]:
import pandas as pd
#################################################################################
# input nodes: 256, data

#downloaded = drive.CreateFile({'id':"1ln3S7u0KhekzydsiezBOkr-ePFNB1aP_"})
#downloaded.GetContentFile('sub_sample_data.csv')
data_xy = pd.read_csv('total_data.csv')

#################################################################################


In [5]:
print('==================================data==================================')
print(data_xy)
print(data_xy.shape)
print(type(data_xy))

==================================data==================================
      ax1  ax2  ax3  ax4  ax5  ax6  ax7  ax8  ax9  ax10  ...  gz12  gz13  \
0     1.6  1.8  1.6  1.5  1.3  1.5  1.2  1.5  1.0   0.9  ...  18.3   7.0   
1    -0.6 -0.8 -1.2 -1.2 -1.4 -1.5 -1.5 -1.6 -1.6  -1.8  ...  12.3   6.4   
2    -2.4 -2.3 -2.5 -2.3 -2.4 -2.4 -2.4 -2.6 -2.7  -2.9  ...  11.8  14.3   
3    -3.4 -3.4 -3.2 -3.4 -0.7 -0.7 -0.6 -0.7 -0.7  -0.7  ...  26.0  10.9   
4    -0.3 -0.6 -0.9 -0.9 -0.9 -0.9 -0.9 -0.9 -0.9  -0.9  ...  -2.9 -14.7   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...   ...  ...   ...   ...   
1881  0.4  0.5  0.6  0.5  0.5  0.5  0.5  0.5  0.5   0.5  ... -10.1  17.8   
1882  0.6  0.5  0.6  0.4  0.6  0.5  0.5  0.5  0.5   0.5  ...   5.0   7.3   
1883  0.6  0.5  0.5  0.6  0.6  0.5  0.6  0.7  0.6   0.5  ...  21.6  28.1   
1884  0.5  0.4  0.5  0.6  0.6  0.6  0.6  0.6  0.5   0.5  ...   8.1  -1.5   
1885  0.5  0.5  0.5  0.5  0.5  0.5  0.4  0.4  0.4   0.4  ...  22.6 -34.5   

      gz14  gz

In [6]:
# 데이터와 라벨 나누기
def split_data(dataset):
    """ 데이터셋 나누기
    
    Parameters
    ----------
    dataset : 데이터셋
    
    Returns
    -------
    (데이터, 레이블)
    """

    total_value = []

    dataset = dataset.transpose()
    value = dataset[:-1]
    label = dataset[-1:]
    label = np.array(label.transpose())
    #label = label.reshape(-1)

    for i in range(len(value.columns)):
        v_list = list(np.array(value[i].tolist()))
        total_value.append(v_list)

    total_value = np.array(total_value)

    return total_value, label

In [7]:
x_data, y_data = split_data(data_xy)

print(x_data)
print(y_data)

[[  1.6   1.8   1.6 ...  25.7  16.8   5.7]
 [ -0.6  -0.8  -1.2 ...   5.4   7.4   6.2]
 [ -2.4  -2.3  -2.5 ...   3.9   3.9   6.5]
 ...
 [  0.6   0.5   0.5 ... -14.9   1.8  -6.3]
 [  0.5   0.4   0.5 ...  24.1   3.3  -5. ]
 [  0.5   0.5   0.5 ...   2.4  -2.9  17.8]]
[[0.]
 [0.]
 [0.]
 ...
 [6.]
 [6.]
 [6.]]


In [8]:
temp_data = (x_data+180)/360
print(str(temp_data.shape))
print(str(temp_data[1,:]))
print(str(temp_data[500,:]))

(1886, 120)
[0.49833333 0.49777778 0.49666667 0.49666667 0.49611111 0.49583333
 0.49583333 0.49555556 0.49555556 0.495      0.49444444 0.49444444
 0.49416667 0.49416667 0.49388889 0.49388889 0.49361111 0.49361111
 0.49388889 0.49361111 0.49       0.48972222 0.49138889 0.49138889
 0.49138889 0.49111111 0.49166667 0.49166667 0.49194444 0.49222222
 0.49194444 0.4925     0.4925     0.49277778 0.49277778 0.49277778
 0.49333333 0.49333333 0.49305556 0.49361111 0.50472222 0.505
 0.50555556 0.50555556 0.50555556 0.50555556 0.50555556 0.50527778
 0.50611111 0.50555556 0.50583333 0.50555556 0.50555556 0.50555556
 0.50555556 0.50583333 0.50555556 0.50555556 0.50583333 0.50555556
 0.50222222 0.5075     0.57555556 0.5275     0.52722222 0.52861111
 0.525      0.51694444 0.52916667 0.50416667 0.51055556 0.50527778
 0.50944444 0.50444444 0.49444444 0.505      0.50444444 0.48777778
 0.49388889 0.49722222 0.585      0.59527778 0.56666667 0.57083333
 0.55027778 0.56194444 0.54666667 0.525      0.53583333

In [9]:
#스케일러
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_data)
x_data = scaler.transform(x_data)

print(type(x_data))
print(type(y_data))
print(y_data)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[0.]
 [0.]
 [0.]
 ...
 [6.]
 [6.]
 [6.]]


In [ ]:
list = np.where(y_data==0)
x_0 = x_data[list[0]]
y_0 = y_data[list[0]]

list = np.where(y_data==1)
x_1 = x_data[list[0]]
y_1 = y_data[list[0]]

list = np.where(y_data==2)
x_2 = x_data[list[0]]
y_2 = y_data[list[0]]

list = np.where(y_data==3)
x_3 = x_data[list[0]]
y_3 = y_data[list[0]]

list = np.where(y_data==4)
x_4 = x_data[list[0]]
y_4 = y_data[list[0]]

list = np.where(y_data==5)
x_5 = x_data[list[0]]
y_5 = y_data[list[0]]

list = np.where(y_data==6)
x_6 = x_data[list[0]]
y_6 = y_data[list[0]]

print(x_4)
print(y_6)

In [11]:
# Shuffle & Split
from sklearn import model_selection
train_x_0, test_x_0, train_y_0, test_y_0 = model_selection.train_test_split(x_0, y_0, test_size=0.25)
train_x_1, test_x_1, train_y_1, test_y_1 = model_selection.train_test_split(x_1, y_1, test_size=0.25)
train_x_2, test_x_2, train_y_2, test_y_2 = model_selection.train_test_split(x_2, y_2, test_size=0.25)
train_x_3, test_x_3, train_y_3, test_y_3 = model_selection.train_test_split(x_3, y_3, test_size=0.25)
train_x_4, test_x_4, train_y_4, test_y_4 = model_selection.train_test_split(x_4, y_4, test_size=0.25)
train_x_5, test_x_5, train_y_5, test_y_5 = model_selection.train_test_split(x_5, y_5, test_size=0.25)
train_x_6, test_x_6, train_y_6, test_y_6 = model_selection.train_test_split(x_6, y_6, test_size=0.25)

In [ ]:
'''
train_y_0 = train_y_0[:, np.newaxis]
train_y_1 = train_y_1[:, np.newaxis]
train_y_2 = train_y_2[:, np.newaxis]
train_y_3 = train_y_3[:, np.newaxis]
train_y_4 = train_y_4[:, np.newaxis]
train_y_5 = train_y_5[:, np.newaxis]
train_y_6 = train_y_6[:, np.newaxis]
train_y_7 = train_y_7[:, np.newaxis]
test_y_0 = test_y_0[:, np.newaxis]
test_y_1 = test_y_1[:, np.newaxis]
test_y_2 = test_y_2[:, np.newaxis]
test_y_3 = test_y_3[:, np.newaxis]
test_y_4 = test_y_4[:, np.newaxis]
test_y_5 = test_y_5[:, np.newaxis]
test_y_6 = test_y_6[:, np.newaxis]
test_y_7 = test_y_7[:, np.newaxis]
'''

'\ntrain_y_0 = train_y_0[:, np.newaxis]\ntrain_y_1 = train_y_1[:, np.newaxis]\ntrain_y_2 = train_y_2[:, np.newaxis]\ntrain_y_3 = train_y_3[:, np.newaxis]\ntrain_y_4 = train_y_4[:, np.newaxis]\ntrain_y_5 = train_y_5[:, np.newaxis]\ntrain_y_6 = train_y_6[:, np.newaxis]\ntrain_y_7 = train_y_7[:, np.newaxis]\ntest_y_0 = test_y_0[:, np.newaxis]\ntest_y_1 = test_y_1[:, np.newaxis]\ntest_y_2 = test_y_2[:, np.newaxis]\ntest_y_3 = test_y_3[:, np.newaxis]\ntest_y_4 = test_y_4[:, np.newaxis]\ntest_y_5 = test_y_5[:, np.newaxis]\ntest_y_6 = test_y_6[:, np.newaxis]\ntest_y_7 = test_y_7[:, np.newaxis]\n'

In [12]:
train_0 = np.append(train_x_0, train_y_0, axis=1)
train_1 = np.append(train_x_1, train_y_1, axis=1)
train_2 = np.append(train_x_2, train_y_2, axis=1)
train_3 = np.append(train_x_3, train_y_3, axis=1)
train_4 = np.append(train_x_4, train_y_4, axis=1)
train_5 = np.append(train_x_5, train_y_5, axis=1)
train_6 = np.append(train_x_6, train_y_6, axis=1)

print(train_3)

[[0.49074074 0.36190476 0.54032258 ... 0.50394011 0.43156863 3.        ]
 [0.51851852 0.58095238 0.5        ... 0.36485422 0.49215686 3.        ]
 [0.5462963  0.59047619 0.55645161 ... 0.5037431  0.67490196 3.        ]
 ...
 [0.59259259 0.58095238 0.54032258 ... 0.61643026 0.48470588 3.        ]
 [0.43518519 0.38095238 0.33064516 ... 0.51851852 0.60137255 3.        ]
 [0.55555556 0.61904762 0.53225806 ... 0.57880221 0.38470588 3.        ]]


In [13]:
train_xy = np.append(train_0, train_1, axis=0)
train_xy = np.append(train_xy, train_2, axis=0)
train_xy = np.append(train_xy, train_3, axis=0)
train_xy = np.append(train_xy, train_4, axis=0)
train_xy = np.append(train_xy, train_5, axis=0)
train_xy = np.append(train_xy, train_6, axis=0)

print(train_xy)
print(train_xy.shape)

[[0.64814815 0.68571429 0.64516129 ... 0.53506698 0.47980392 0.        ]
 [0.58333333 0.57142857 0.50806452 ... 0.50610717 0.52117647 0.        ]
 [0.40740741 0.40952381 0.40322581 ... 0.51930654 0.50647059 0.        ]
 ...
 [0.62962963 0.63809524 0.58870968 ... 0.52167061 0.46784314 6.        ]
 [0.62037037 0.62857143 0.58064516 ... 0.46591805 0.50705882 6.        ]
 [0.62037037 0.62857143 0.58064516 ... 0.50197006 0.4845098  6.        ]]
(1413, 121)


In [14]:
test_0 = np.append(test_x_0, test_y_0, axis=1)
test_1 = np.append(test_x_1, test_y_1, axis=1)
test_2 = np.append(test_x_2, test_y_2, axis=1)
test_3 = np.append(test_x_3, test_y_3, axis=1)
test_4 = np.append(test_x_4, test_y_4, axis=1)
test_5 = np.append(test_x_5, test_y_5, axis=1)
test_6 = np.append(test_x_6, test_y_6, axis=1)

print(test_1)
print(test_1.shape)

[[0.35185185 0.35238095 0.34677419 ... 0.50394011 0.50117647 1.        ]
 [0.55555556 0.57142857 0.52419355 ... 0.50610717 0.49960784 1.        ]
 [0.58333333 0.59047619 0.55645161 ... 0.5072892  0.50607843 1.        ]
 ...
 [0.67592593 0.68571429 0.62903226 ... 0.50669819 0.5054902  1.        ]
 [0.58333333 0.59047619 0.55645161 ... 0.5072892  0.50607843 1.        ]
 [0.26851852 0.26666667 0.26612903 ... 0.51615445 0.52117647 1.        ]]
(161, 121)


In [15]:
test_xy = np.append(test_0, test_1, axis=0)
test_xy = np.append(test_xy, test_2, axis=0)
test_xy = np.append(test_xy, test_3, axis=0)
test_xy = np.append(test_xy, test_4, axis=0)
test_xy = np.append(test_xy, test_5, axis=0)
test_xy = np.append(test_xy, test_6, axis=0)

print(test_xy)
print(test_xy.shape)

[[0.57407407 0.57142857 0.53225806 ... 0.49763593 0.49568627 0.        ]
 [0.67592593 0.6952381  0.64516129 ... 0.50886525 0.51647059 0.        ]
 [0.66666667 0.67619048 0.62096774 ... 0.39736013 0.52117647 0.        ]
 ...
 [0.60185185 0.60952381 0.57258065 ... 0.51457841 0.49509804 6.        ]
 [0.61111111 0.61904762 0.58064516 ... 0.54255319 0.52372549 6.        ]
 [0.62037037 0.63809524 0.58870968 ... 0.51654846 0.51803922 6.        ]]
(473, 121)


In [16]:
train_xy = pd.DataFrame(train_xy)
test_xy = pd.DataFrame(test_xy)

print(train_xy)
print(test_xy)
print(type(train_xy))
print(type(test_xy))

           0         1         2         3         4         5         6    \
0     0.648148  0.685714  0.645161  0.721739  0.692982  0.783505  0.787037   
1     0.583333  0.571429  0.508065  0.591304  0.561404  0.628866  0.638889   
2     0.407407  0.409524  0.403226  0.452174  0.429825  0.474227  0.490741   
3     0.675926  0.704762  0.637097  0.721739  0.701754  0.783505  0.777778   
4     0.546296  0.552381  0.516129  0.582609  0.561404  0.628866  0.648148   
...        ...       ...       ...       ...       ...       ...       ...   
1408  0.611111  0.609524  0.564516  0.626087  0.605263  0.670103  0.666667   
1409  0.666667  0.676190  0.620968  0.704348  0.675439  0.773196  0.759259   
1410  0.629630  0.638095  0.588710  0.660870  0.640351  0.721649  0.722222   
1411  0.620370  0.628571  0.580645  0.652174  0.631579  0.711340  0.712963   
1412  0.620370  0.628571  0.580645  0.652174  0.631579  0.711340  0.712963   

       7         8         9    ...       111       112       1

In [19]:
train_xy.to_csv('./train_data.csv', index=None)

In [20]:
test_xy.to_csv('./test_data.csv', index=None)

In [21]:
!pwd

/content
